### HTML 데이터 파싱을 이용한 크롤링
- 네이버 실시간 검색어 순위 
    - HTML 포멧의 문자열을 가져오는 URL 찾음
    - 요청 -> 응답(HTML 포멧(str)의 데이터를 저장)
    - 데이터 파싱
        - bs4 패키지의 BeautifulSoup 클래스를 이용해서 dom 객체를 생성
        - dom 객체에서 우리가 원하는 데이터를 css-selector를 이용해서 수집
        - 데이터 프레임으로 만들기

### 네이버 연관검색어 가져오기

In [2]:
import requests
from bs4 import BeautifulSoup

In [6]:
# 1. requests 이용하여 HTML 코드 문자열 데이터 가져오기

query = "삼성전자"
url = "https://search.naver.com/search.naver?\
sm=top_hty&fbm=1&ie=utf8&query={}".format(query)
print(url)

https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=삼성전자


In [17]:
response = requests.get(url)
response

<Response [200]>

In [18]:
# 2. BeautifulSoup을 이용하여 css selector를 사용할수 있는 객체로 파싱
dom = BeautifulSoup(response.content, "html.parser")

In [19]:
# 3. css selector를 이용하여 원하는 데이터 수집
# select : 여러개의 엘리먼트 객체들을 리스트로 가져옴
# select_one : 하나의 엘리먼트 객체를 가져옴

elements = dom.select("._related_keyword_ul > li")
len(elements) #len 찍어보면 몇개인지 알 수 있으니까

# copy selector를 사용할 수 있음
# copy selector를 사용하면 id가 나올 때까지 찾아주기 때문에 약가 긴편


10

In [23]:
type(elements[0]), elements[0], elements[0].text.strip()

(bs4.element.Tag,
 <li> <a data-area="*q" data-idx="1" href="?where=nexearch&amp;query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%9A%B0&amp;ie=utf8&amp;sm=tab_she&amp;qdt=0">삼성전자우</a> </li>,
 '삼성전자우')

In [24]:
# keywords = []
# for i in range(len(elements)):
#     keywords.append(elements[i].text.strip())


# print(keywords)  

# 이런 코드는 좋은 코드가 아님

['삼성전자우', '카카오', 'sk하이닉스', '삼성', '현대자동차', '삼성전자서비스센터', 'lg화학', '네이버', '코스피', '삼성sdi']


In [25]:
# list comprehenshion을 사용하도록 하자
[element.text.strip() for element in elements]

['삼성전자우',
 '카카오',
 'sk하이닉스',
 '삼성',
 '현대자동차',
 '삼성전자서비스센터',
 'lg화학',
 '네이버',
 '코스피',
 '삼성sdi']

In [26]:
# 함수로 만들어 봅시다

def linked_words(query):
    url = "https://search.naver.com/search.naver?\
    sm=top_hty&fbm=1&ie=utf8&query={}".format(query)
    
    response = requests.get(url)    
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select("._related_keyword_ul > li")
    
    return [element.text.strip() for element in elements]

In [27]:
linked_words("커피머신")

['와인에빠진귤친구들',
 '반자동커피머신',
 '스페인하숙커피머신',
 '가정용에스프레소머신추천',
 '카페박람회',
 '캡슐커피',
 '가정용커피머신추천',
 '스페인하숙커피',
 '전자동커피머신',
 '커피메이커']

### 다음 기사 가져오기
- 다음 뉴스 메인 페이지 15개의 기사 제목, 링크, 내용 가져오기

In [111]:
url = "https://daum.net"
response = requests.get(url)
dom = BeautifulSoup(response.content, "html.parser")
elements = dom.select(".list_txt > li")
len(elements), elements[0]

(15,
 <li>
 <a class="link_txt " data-tiara-action-kind="ClickContent" data-tiara-action-name="뉴스1탭_텍스트소재" data-tiara-custom="c_id=hamny-20200824125600267" data-tiara-id="hamny-20200824125600267" data-tiara-imp_id="" data-tiara-layer="txt" data-tiara-ordnum="1" href="https://v.daum.net/v/20200824125600267">박능후 "사랑제일교회 협조 있었다면 확진자 상당히 줄었을 것"</a>
 </li>)

In [112]:
datas = []

for element in elements:
    datas.append({
        "title": element.text.strip(),
        "link" : element.select_one('a').get("href")  # a 태그 안에서 속성 값 href를 가져오겠다는 것
    })

In [113]:
news = pd.DataFrame(datas)
news

,title,link
0,"박능후 ""사랑제일교회 협조 있었다면 확진자 상당히 줄었을 것""",https://v.daum.net/v/20200824125600267
1,'공백 최소화'라더니..수술 연기에 우는 환자들,https://v.daum.net/v/20200824125100170
2,국보 불상 2점은 어찌되나..보물 2점 국립박물관에 판 간송재단의 다음 행보는?,https://v.daum.net/v/20200824124745113
3,"감염학회 ""코로나19 급증, 거리두기 3단계 격상 불가피""",https://v.daum.net/v/20200824124121024
4,정규직 늘리랬더니 퇴직금잔치?..고용정보원 퇴직금 충당 10배 늘어,https://v.daum.net/v/20200824123900994
5,검찰 중간간부 인사 임박..형사·공판부 우대 이어갈듯,https://v.daum.net/v/20200824123726983
6,"일본 ""지소미아 역내 평화에 기여..안정적 운용 중요""",https://v.daum.net/v/20200824123554967
7,서울 시내버스 기사 3명 확진..5618번·6512번 지연운행 논의,https://v.daum.net/v/20200824123434948
8,상 받고도 부끄러워하는 중국 지방 공무원 '달팽이상',https://v.daum.net/v/20200824123409940
9,거리두기 3단계 격상땐 실내외 10인이상 모임 금지..등교수업 중단,https://v.daum.net/v/20200824123206900


In [123]:
def get_content(link):
    response = requests.get(link)
    dom = BeautifulSoup(response.content, "html.parser")
    divs = dom.select("#harmonyContainer > section > p")[1:-2]
    return "".join([div.text for div in divs])

In [124]:
news["content"] = news["link"].apply(get_content)

In [126]:
news

,title,link,content
0,"박능후 ""사랑제일교회 협조 있었다면 확진자 상당히 줄었을 것""",https://v.daum.net/v/20200824125600267,박 장관은 이날 오전 2019년 회계연도 결산을 위해 열린 국회 예산결산특별위원회 ...
1,'공백 최소화'라더니..수술 연기에 우는 환자들,https://v.daum.net/v/20200824125100170,하지만 집단휴진이 신종 코로나바이러스 감염증(코로나19)의 전국적인 재확산과 맞물리...
2,국보 불상 2점은 어찌되나..보물 2점 국립박물관에 판 간송재단의 다음 행보는?,https://v.daum.net/v/20200824124745113,국보 불상 2점은 어찌되나. 지난 5월 간송 전형필(1906~1962)의 후손이 경...
3,"감염학회 ""코로나19 급증, 거리두기 3단계 격상 불가피""",https://v.daum.net/v/20200824124121024,"학회는 24일 성명을 통해 ""지난 23일 0시를 기준으로 전국적인 사회적 거리두기 ..."
4,정규직 늘리랬더니 퇴직금잔치?..고용정보원 퇴직금 충당 10배 늘어,https://v.daum.net/v/20200824123900994,"하지만 이종배 의원이 8월 결산국회를 맞아 분석한 바에 따르면, 고용정보원이 작년 ..."
5,검찰 중간간부 인사 임박..형사·공판부 우대 이어갈듯,https://v.daum.net/v/20200824123726983,법무부는 24일 오전 10시 정부과천청사에서 검찰인사위 회의를 열었다. 이날 회의에...
6,"일본 ""지소미아 역내 평화에 기여..안정적 운용 중요""",https://v.daum.net/v/20200824123554967,일본 정부 대변인인 스가 요시히데(菅義偉) 관방장관은 이날 정례 브리핑에서 한국 정...
7,서울 시내버스 기사 3명 확진..5618번·6512번 지연운행 논의,https://v.daum.net/v/20200824123434948,24일 서울시에 따르면 보성운수 소속 버스 기사 3명이 21∼23일 잇따라 코로나1...
8,상 받고도 부끄러워하는 중국 지방 공무원 '달팽이상',https://v.daum.net/v/20200824123409940,다른 시상식과 달리 수상자들은 상을 받을 때 고개를 들지 못했다. 업무 추진이 너무...
9,거리두기 3단계 격상땐 실내외 10인이상 모임 금지..등교수업 중단,https://v.daum.net/v/20200824123206900,윤태호 보건복지부 중앙사고수습본부 방역총괄반장은 24일 중앙재난안전대책본부 정례 브...


### Gmarket 베스트 상품 200개 크롤링
- 상품명, 원가, 판매가
- 썸네일 이미지

In [147]:
response = requests.get("http://corners.gmarket.co.kr/Bestsellers")
dom = BeautifulSoup(response.content, "html.parser")

In [159]:
selector = "#gBestWrap > div > div:nth-of-type(5) > div:nth-of-type(3) > ul > li"
elements = dom.select(selector)

In [160]:
len(elements)

0

In [ ]:
# stream = True : 데이터를 가져올 때 잘게 쪼개서 가져온다는 것

#### Quiz
- 다음 실시간 검색어 순위 데이터 수집
- https://daum.net
- 네이버 실시간 검색어와 다음 실시간 검색어 중복 키워드 출력

In [28]:
response = requests.get("https://daum.net")
response

<Response [200]>

In [30]:
dom = BeautifulSoup(response.content, "html.parser")

In [33]:
elements = dom.select(".realtime_part > .list_hotissue.issue_row > li")
len(elements)

10

In [35]:
rank = elements[0].select_one(".ir_wa").text
keyword = elements[0].select_one(".link_issue").text
print(rank, keyword)

1위 수능 시간표


In [36]:
datas = []

for element in elements:
    datas.append({
        "rank": element.select_one(".ir_wa").text,
        "keyword": element.select_one(".link_issue").text,
    })
    
daum_keywords = pd.DataFrame(datas)
daum_keywords.tail(2)

,keyword,rank
8,수능 수학,9위
9,수능 국어,10위


In [38]:
naver_df = naver_keywords()
daum_df = pd.DataFrame(datas)

In [42]:
set(naver_df["keyword"]) & set(daum_df["keyword"])

{'구본영', '김경수', '수능 국어', '수능 수학', '수능 시간표', '수능 영어', '유리 오빠', '이문정'}